<a href="https://colab.research.google.com/github/mehdii190/neural-network/blob/main/src/diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing 

In [149]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

Code 

In [150]:
data = pd.read_csv("/content/csv/diabetes.csv")

In [151]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


info data 

In [152]:
data.drop('DiabetesPedigreeFunction', axis=1, inplace=True)
x = data.iloc[:,0:-1].values
y = data.iloc[:,-1]
print(data.info())
print(x)
print(y)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Pregnancies    768 non-null    int64  
 1   Glucose        768 non-null    int64  
 2   BloodPressure  768 non-null    int64  
 3   SkinThickness  768 non-null    int64  
 4   Insulin        768 non-null    int64  
 5   BMI            768 non-null    float64
 6   Age            768 non-null    int64  
 7   Outcome        768 non-null    int64  
dtypes: float64(1), int64(7)
memory usage: 48.1 KB
None
[[  6.  148.   72.  ...   0.   33.6  50. ]
 [  1.   85.   66.  ...   0.   26.6  31. ]
 [  8.  183.   64.  ...   0.   23.3  32. ]
 ...
 [  5.  121.   72.  ... 112.   26.2  30. ]
 [  1.  126.   60.  ...   0.   30.1  47. ]
 [  1.   93.   70.  ...   0.   30.4  23. ]]
0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: in

In [153]:
x = np.array(x,dtype="float64")
y = np.array(y,dtype="float64")
y.shape
x.shape

(768, 7)

scaler x

In [154]:
sc = StandardScaler()
x = sc.fit_transform(x)

In [155]:
x = torch.tensor(x)
y = torch.tensor(y).unsqueeze(1)

In [156]:
print(x.shape)
print(y.shape)
x.dtype

torch.Size([768, 7])
torch.Size([768, 1])


torch.float64

class dataset ⬇

In [157]:
class Dataset(Dataset):

  def __init__(self,x,y):

    self.x = x

    self.y = y


  def __getitem__(self, index):

    return self.x[index] , self.y[index]

  def __len__(self):
    
    return len(self.x)

In [158]:
dataset = Dataset(x,y)

In [159]:
len(dataset)

768

In [160]:
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True)

In [161]:
print("there is {} batch is dataset".format(len(train_loader)))

for (x,y) in train_loader:
  print("for one interation (batch) , there is : ")
  print("data :  {}".format(x.shape))
  print("label : {}".format(y.shape))
  break

there is 24 batch is dataset
for one interation (batch) , there is : 
data :  torch.Size([32, 7])
label : torch.Size([32, 1])


neural network formula

In [162]:
class Model(nn.Module):

  def __init__(self,input_feature,output_feature):

    super(Model,self).__init__()

    self.fc1 = nn.Linear(input_feature,5)
    self.fc2 = nn.Linear(5,4)
    self.fc3 = nn.Linear(4,3)
    self.fc4 = nn.Linear(3,output_feature)
    self.sigmoid = nn.Sigmoid()
    self.tanh = nn.Tanh()



  def forward(self,x):

    out = self.fc1(x)
    out = self.tanh(out)
    out = self.fc2(out)
    out = self.tanh(out)
    out = self.fc3(out)
    out = self.tanh(out)
    out = self.fc4(out)
    out = self.sigmoid(out)
    return out

In [163]:
net = Model(7,1)


criterion = torch.nn.BCELoss(size_average=True)

optimizer =torch.optim.SGD(net.parameters() , lr=0.1 , momentum=0.9)

/usr/local/lib/python3.9/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


predict

In [165]:
epochs = 200


for  epoch in range(200):
  for inputs, labels in train_loader:

    inputs = inputs.float()

    labels = labels.float()

    outputs = net(inputs)

    loss = criterion(outputs,labels)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()


  output = (outputs > 0.5).float()

  accuracy = (output == labels).float().mean()

  print("Epoch {}/{}, Loss : {:.3f} , Accuracy : {:.3f}".format(epoch + 1,epochs, loss, accuracy))
  print("\n")



Epoch 1/200, Loss : 0.337 , Accuracy : 0.844


Epoch 2/200, Loss : 0.426 , Accuracy : 0.781


Epoch 3/200, Loss : 0.352 , Accuracy : 0.812


Epoch 4/200, Loss : 0.314 , Accuracy : 0.875


Epoch 5/200, Loss : 0.558 , Accuracy : 0.781


Epoch 6/200, Loss : 0.363 , Accuracy : 0.750


Epoch 7/200, Loss : 0.309 , Accuracy : 0.875


Epoch 8/200, Loss : 0.471 , Accuracy : 0.719


Epoch 9/200, Loss : 0.391 , Accuracy : 0.750


Epoch 10/200, Loss : 0.266 , Accuracy : 0.938


Epoch 11/200, Loss : 0.448 , Accuracy : 0.688


Epoch 12/200, Loss : 0.443 , Accuracy : 0.750


Epoch 13/200, Loss : 0.516 , Accuracy : 0.781


Epoch 14/200, Loss : 0.313 , Accuracy : 0.906


Epoch 15/200, Loss : 0.368 , Accuracy : 0.750


Epoch 16/200, Loss : 0.404 , Accuracy : 0.844


Epoch 17/200, Loss : 0.295 , Accuracy : 0.875


Epoch 18/200, Loss : 0.516 , Accuracy : 0.812


Epoch 19/200, Loss : 0.323 , Accuracy : 0.781


Epoch 20/200, Loss : 0.481 , Accuracy : 0.781


Epoch 21/200, Loss : 0.427 , Accuracy : 0.844


E